# GSE106582

# GSE21815

# GSE44076

# GSE8671

# Now Tuning

# Grid Serach CV GSE106582

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier,
    AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier
)
from xgboost import XGBClassifier
import warnings

warnings.filterwarnings("ignore")

# Load datasets
df_train = pd.read_csv("ML_DEG_dataset.csv")
columns_of_interest = ['PTRF', 'PRNP', 'FLNA', 'SPARCL1', 'RHOB', 'PALLD', 'SNCA',
                       'CSRP1', 'TNS1', 'DPYSL3', 'TIMP2', 'MEF2C', 'ILK', 'ACTN1', 'TGFB1I1', 'RHOQ', 'CAV2','target']
df_train = df_train[columns_of_interest]
#df_train = df_train.iloc[:, 1:]
df_train = df_train[[col for col in df_train.columns if col != 'target'] + ['target']]
X_train = df_train.iloc[:, 0:-1]
y_train = df_train.iloc[:, -1]

df_test = pd.read_csv("GSE106582_gene_expression_renamed.csv")
df_test = df_test.iloc[:, 1:]
df_test = df_test[[col for col in df_test.columns if col != 'target'] + ['target']]
X_test = df_test.iloc[:, 0:-1]
y_test = df_test.iloc[:, -1]

# Ensure common features in both train and test sets
common_cols = set(X_train.columns).intersection(X_test.columns)
X_train_common = X_train[list(common_cols)]
X_test_common = X_test[list(common_cols)]

# Hyperparameter grid for each model
param_grids = {
    'Logistic Regression': {
        'C': [0.01, 0.1, 1, 10, 100],
        'penalty': ['l1', 'l2', 'elasticnet'],
        'solver': ['saga'],
        'max_iter': [500]
    },
    'K-Nearest Neighbors': {
        'n_neighbors': [3, 5, 7, 9],
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan']
    },
    'Random Forest': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    },
    'Gradient Boosting': {
        'n_estimators': [100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 1.0],
        'min_samples_split': [2, 5]
    },
    'AdaBoost': {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 1]
    },
    'Bagging': {
        'n_estimators': [50, 100, 200],
        'max_samples': [0.5, 0.75, 1.0],
        'max_features': [0.5, 0.75, 1.0],
        'bootstrap': [True, False]
    },
    'Extra Trees': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    },
     'XGBoost': {
        'n_estimators': [100, 200, 500],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    }
}

# Define models
models = {
    'Logistic Regression': LogisticRegression(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Bagging': BaggingClassifier(),
    'Extra Trees': ExtraTreesClassifier(),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

# Dictionary to store best parameters
best_params = {}

# Tune and evaluate models
for name, model in models.items():
    print(f"Tuning {name}...")

    # Perform GridSearchCV
    grid_search = GridSearchCV(model, param_grids[name], cv=10, n_jobs=-1, scoring='accuracy')
    grid_search.fit(X_train_common, y_train)

    # Store best parameters
    best_params[name] = grid_search.best_params_
    print(f"Best parameters for {name}: {grid_search.best_params_}\n")

    # Train model with best parameters
    best_model = grid_search.best_estimator_
    best_model.fit(X_train_common, y_train)

    # Predict on test set
    predictions = best_model.predict(X_test_common)

    # Compute metrics
    test_accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted', zero_division=0)
    recall = recall_score(y_test, predictions, average='weighted', zero_division=0)
    f1 = f1_score(y_test, predictions, average='weighted', zero_division=0)
    mcc = matthews_corrcoef(y_test, predictions)

    print(f"{name} Testing Accuracy: {test_accuracy * 100:.2f}%")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-Score: {f1:.2f}")
    print(f"MCC: {mcc:.2f}")

    # Print confusion matrix and classification report
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, predictions))
    print("Classification Report:")
    print(classification_report(y_test, predictions))
    print("-" * 60)

# Print final best parameters
print("\nFinal Best Parameters for All Models:")
for model, params in best_params.items():
    print(f"{model}: {params}")

Tuning Logistic Regression...
Best parameters for Logistic Regression: {'C': 0.01, 'max_iter': 500, 'penalty': 'l1', 'solver': 'saga'}

Logistic Regression Testing Accuracy: 52.58%
Precision: 0.76
Recall: 0.53
F1-Score: 0.47
MCC: 0.30
Confusion Matrix:
[[26 91]
 [ 1 76]]
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.22      0.36       117
           1       0.46      0.99      0.62        77

    accuracy                           0.53       194
   macro avg       0.71      0.60      0.49       194
weighted avg       0.76      0.53      0.47       194

------------------------------------------------------------
Tuning K-Nearest Neighbors...
Best parameters for K-Nearest Neighbors: {'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'uniform'}

K-Nearest Neighbors Testing Accuracy: 47.94%
Precision: 0.64
Recall: 0.48
F1-Score: 0.41
MCC: 0.15
Confusion Matrix:
[[22 95]
 [ 6 71]]
Classification Report:
              precisio

In [9]:
df_train.isnull()


,PTRF,PRNP,FLNA,SPARCL1,RHOB,PALLD,SNCA,CSRP1,TNS1,DPYSL3,TIMP2,MEF2C,ILK,ACTN1,TGFB1I1,RHOQ,CAV2,target
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
341,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
342,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
343,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [10]:
df_train.isnull().any()

,0
PTRF,False
PRNP,False
FLNA,False
SPARCL1,False
RHOB,False
PALLD,False
SNCA,False
CSRP1,False
TNS1,False
DPYSL3,False


In [11]:
df_train.columns[df_train.isnull().any()]


Index([], dtype='object')

In [5]:
X_test_common.isnull().sum()

,0
TGFB1I1,1
ILK,1
PRNP,1
FLNA,0
PTRF,1
PALLD,0
CSRP1,1
ACTN1,1
CAV2,0
SNCA,0


In [2]:
X_train_common

,TGFB1I1,ILK,PRNP,FLNA,PTRF,PALLD,CSRP1,ACTN1,CAV2,SNCA,SPARCL1,RHOB,TIMP2,MEF2C,TNS1,RHOQ,DPYSL3
0,128.91800,1334.019000,2525.33300,129.41580,1815.16700,1638.3070,2751.51100,1302.68100,126.19610,83.461750,1551.67000,2217.573000,2282.42000,393.864700,34.813670,1656.344000,855.154400
1,146.04050,987.597000,2478.02200,201.83130,2654.56600,1938.3600,3448.62500,2296.27400,231.37260,106.309100,1086.06200,1288.359000,1225.11500,123.796300,55.187360,2055.204000,1548.034000
2,1500.77200,7076.318000,9752.85100,4744.04400,28658.69000,13296.1000,33181.18000,13146.80000,731.82810,423.809000,14092.56000,8771.118000,9982.55900,555.490400,770.330300,6837.925000,22552.190000
3,81.53146,806.585600,3518.65300,117.58670,1306.54900,1377.7610,2822.32000,2180.37600,123.91260,51.109720,436.62330,4295.779000,921.19460,45.465650,29.580640,749.293500,408.538600
4,689.19950,3120.280000,3504.77600,2481.00300,19209.36000,7848.7430,21789.32000,8109.85700,297.20290,161.920600,5941.94700,3388.500000,4066.77300,391.786400,240.026600,2012.708000,7205.648000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,107.19923,122.380936,545.56420,157.54823,151.43648,708.5136,231.58870,1126.93410,117.67158,11.782273,92.23398,78.066880,404.02570,24.651146,27.482962,16.600166,22.743372
341,297.19640,126.028010,981.79990,718.72160,432.79465,1053.3113,482.59622,908.68695,191.21483,11.031474,639.76310,123.509910,593.02783,68.284325,43.319565,64.263370,124.817505
342,217.49037,102.200990,550.05050,236.23875,210.38971,946.1038,235.60060,798.90920,301.34503,11.946541,220.10430,92.778730,682.45400,56.451244,30.474731,28.831093,131.724150
343,169.52666,91.976770,1805.44070,330.13992,230.68983,642.6224,201.74535,751.56934,140.47680,9.838649,587.13130,117.208885,741.87305,77.262535,32.156227,35.960430,102.946260


In [3]:
X_train_common.isnull().sum()

,0
TGFB1I1,0
ILK,0
PRNP,0
FLNA,0
PTRF,0
PALLD,0
CSRP1,0
ACTN1,0
CAV2,0
SNCA,0


# Random Search CV GSE106582

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier,
    AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier
)
from xgboost import XGBClassifier
import warnings

warnings.filterwarnings("ignore")

# Load datasets
df_train = pd.read_csv("ML_DEG_dataset.csv")
columns_of_interest = ['PTRF', 'PRNP', 'FLNA', 'SPARCL1', 'RHOB', 'PALLD', 'SNCA', 'CSRP1', 'TNS1',
                       'DPYSL3', 'TIMP2', 'MEF2C', 'ILK', 'ACTN1', 'TGFB1I1', 'RHOQ', 'CAV2','target']
df_train = df_train[columns_of_interest]
#df_train = df_train.iloc[:, 1:]
df_train = df_train[[col for col in df_train.columns if col != 'target'] + ['target']]
X_train = df_train.iloc[:, 0:-1]
y_train = df_train.iloc[:, -1]

df_test = pd.read_csv("GSE106582_gene_expression_renamed.csv")
df_test = df_test.iloc[:, 1:]
df_test = df_test[[col for col in df_test.columns if col != 'target'] + ['target']]
X_test = df_test.iloc[:, 0:-1]
y_test = df_test.iloc[:, -1]

# Ensure common features in both train and test sets
common_cols = set(X_train.columns).intersection(X_test.columns)
X_train_common = X_train[list(common_cols)]
X_test_common = X_test[list(common_cols)]

# Hyperparameter grid for each model
param_grids = {
    'Logistic Regression': {
        'C': np.logspace(-3, 2, 10),
        'penalty': ['l1', 'l2', 'elasticnet'],
        'solver': ['saga'],
        'max_iter': [500, 1000, 5000],
        'l1_ratio': np.linspace(0, 1, 5).tolist()
    },
    'K-Nearest Neighbors': {
        'n_neighbors': range(3, 15),
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan']
    },
    'Random Forest': {
        'n_estimators': range(100, 501, 100),
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    },
    'Gradient Boosting': {
        'n_estimators': range(100, 501, 100),
        'learning_rate': np.linspace(0.01, 0.2, 5),
        'max_depth': [3, 5, 7, 10],
        'subsample': [0.6, 0.8, 1.0],
        'min_samples_split': [2, 5, 10]
    },
    'AdaBoost': {
        'n_estimators': range(50, 301, 50),
        'learning_rate': np.linspace(0.01, 1, 5)
    },
    'Bagging': {
        'n_estimators': range(50, 301, 50),
        'max_samples': np.linspace(0.5, 1.0, 5),
        'max_features': np.linspace(0.5, 1.0, 5),
        'bootstrap': [True, False]
    },
    'Extra Trees': {
        'n_estimators': range(100, 501, 100),
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    },
    'XGBoost': {
        'n_estimators': [100, 200, 500],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    }
}

# Define models
models = {
    'Logistic Regression': LogisticRegression(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Bagging': BaggingClassifier(),
    'Extra Trees': ExtraTreesClassifier(),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss')

}

# Dictionary to store best parameters
best_params = {}

# Tune and evaluate models using RandomizedSearchCV
for name, model in models.items():
    print(f"Tuning {name}...")

    # Perform RandomizedSearchCV
    random_search = RandomizedSearchCV(model, param_grids[name],
                                       n_iter=20, cv=10,
                                       n_jobs=-1, scoring='accuracy', random_state=7)

    random_search.fit(X_train_common, y_train)

    # Store best parameters
    best_params[name] = random_search.best_params_
    print(f"Best parameters for {name}: {random_search.best_params_}\n")

    # Train model with best parameters
    best_model = random_search.best_estimator_
    best_model.fit(X_train_common, y_train)

    # Predict on test set
    predictions = best_model.predict(X_test_common)

    # Compute metrics
    test_accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted', zero_division=0)
    recall = recall_score(y_test, predictions, average='weighted', zero_division=0)
    f1 = f1_score(y_test, predictions, average='weighted', zero_division=0)
    mcc = matthews_corrcoef(y_test, predictions)

    print(f"{name} Testing Accuracy: {test_accuracy * 100:.2f}%")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-Score: {f1:.2f}")
    print(f"MCC: {mcc:.2f}")

    # Print confusion matrix and classification report
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, predictions))
    print("Classification Report:")
    print(classification_report(y_test, predictions))
    print("-" * 60)

# Print final best parameters
print("\nFinal Best Parameters for All Models:")
for model, params in best_params.items():
    print(f"{model}: {params}")

Tuning Logistic Regression...
Best parameters for Logistic Regression: {'solver': 'saga', 'penalty': 'l1', 'max_iter': 5000, 'l1_ratio': 0.5, 'C': np.float64(0.1668100537200059)}

Logistic Regression Testing Accuracy: 87.11%
Precision: 0.88
Recall: 0.87
F1-Score: 0.87
MCC: 0.74
Confusion Matrix:
[[114   3]
 [ 22  55]]
Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.97      0.90       117
           1       0.95      0.71      0.81        77

    accuracy                           0.87       194
   macro avg       0.89      0.84      0.86       194
weighted avg       0.88      0.87      0.87       194

------------------------------------------------------------
Tuning K-Nearest Neighbors...
Best parameters for K-Nearest Neighbors: {'weights': 'uniform', 'n_neighbors': 3, 'metric': 'manhattan'}

K-Nearest Neighbors Testing Accuracy: 47.94%
Precision: 0.64
Recall: 0.48
F1-Score: 0.41
MCC: 0.15
Confusion Matrix:
[[22 95]
 [ 6 71

# GSE21815 Grid Search CV

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier,
    AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier
)
from xgboost import XGBClassifier
import warnings

warnings.filterwarnings("ignore")

# Load datasets
df_train = pd.read_csv("ML_DEG_dataset.csv")
columns_of_interest = ['PTRF', 'PRNP', 'FLNA', 'SPARCL1', 'RHOB', 'PALLD', 'SNCA', 'CSRP1',
                       'TNS1', 'DPYSL3', 'TIMP2', 'MEF2C', 'ILK', 'ACTN1', 'TGFB1I1', 'RHOQ', 'CAV2','target']
df_train = df_train[columns_of_interest]
#df_train = df_train.iloc[:, 1:]
df_train = df_train[[col for col in df_train.columns if col != 'target'] + ['target']]
X_train = df_train.iloc[:, 0:-1]
y_train = df_train.iloc[:, -1]

df_test = pd.read_csv("GSE21815_gene_expression_renamed.csv")
df_test = df_test.iloc[:, 1:]
df_test = df_test[[col for col in df_test.columns if col != 'target'] + ['target']]
X_test = df_test.iloc[:, 0:-1]
y_test = df_test.iloc[:, -1]

# Ensure common features in both train and test sets
common_cols = set(X_train.columns).intersection(X_test.columns)
X_train_common = X_train[list(common_cols)]
X_test_common = X_test[list(common_cols)]

# Hyperparameter grid for each model
param_grids = {
    'Logistic Regression': {
        'C': [0.01, 0.1, 1, 10, 100],
        'penalty': ['l1', 'l2', 'elasticnet'],
        'solver': ['saga'],
        'max_iter': [500]
    },
    'K-Nearest Neighbors': {
        'n_neighbors': [3, 5, 7, 9],
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan']
    },
    'Random Forest': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    },
    'Gradient Boosting': {
        'n_estimators': [100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 1.0],
        'min_samples_split': [2, 5]
    },
    'AdaBoost': {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 1]
    },
    'Bagging': {
        'n_estimators': [50, 100, 200],
        'max_samples': [0.5, 0.75, 1.0],
        'max_features': [0.5, 0.75, 1.0],
        'bootstrap': [True, False]
    },
    'Extra Trees': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    },
     'XGBoost': {
        'n_estimators': [100, 200, 500],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    }
}

# Define models
models = {
    'Logistic Regression': LogisticRegression(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Bagging': BaggingClassifier(),
    'Extra Trees': ExtraTreesClassifier(),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

# Dictionary to store best parameters
best_params = {}

# Tune and evaluate models
for name, model in models.items():
    print(f"Tuning {name}...")

    # Perform GridSearchCV
    grid_search = GridSearchCV(model, param_grids[name], cv=10, n_jobs=-1, scoring='accuracy')
    grid_search.fit(X_train_common, y_train)

    # Store best parameters
    best_params[name] = grid_search.best_params_
    print(f"Best parameters for {name}: {grid_search.best_params_}\n")

    # Train model with best parameters
    best_model = grid_search.best_estimator_
    best_model.fit(X_train_common, y_train)

    # Predict on test set
    predictions = best_model.predict(X_test_common)

    # Compute metrics
    test_accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted', zero_division=0)
    recall = recall_score(y_test, predictions, average='weighted', zero_division=0)
    f1 = f1_score(y_test, predictions, average='weighted', zero_division=0)
    mcc = matthews_corrcoef(y_test, predictions)

    print(f"{name} Testing Accuracy: {test_accuracy * 100:.2f}%")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-Score: {f1:.2f}")
    print(f"MCC: {mcc:.2f}")

    # Print confusion matrix and classification report
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, predictions))
    print("Classification Report:")
    print(classification_report(y_test, predictions))
    print("-" * 60)

# Print final best parameters
print("\nFinal Best Parameters for All Models:")
for model, params in best_params.items():
    print(f"{model}: {params}")

Tuning Logistic Regression...
Best parameters for Logistic Regression: {'C': 0.01, 'max_iter': 500, 'penalty': 'l1', 'solver': 'saga'}

Logistic Regression Testing Accuracy: 95.74%
Precision: 0.95
Recall: 0.96
F1-Score: 0.95
MCC: 0.61
Confusion Matrix:
[[  5   4]
 [  2 130]]
Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.56      0.62         9
           1       0.97      0.98      0.98       132

    accuracy                           0.96       141
   macro avg       0.84      0.77      0.80       141
weighted avg       0.95      0.96      0.95       141

------------------------------------------------------------
Tuning K-Nearest Neighbors...
Best parameters for K-Nearest Neighbors: {'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'uniform'}

K-Nearest Neighbors Testing Accuracy: 81.56%
Precision: 0.90
Recall: 0.82
F1-Score: 0.85
MCC: 0.12
Confusion Matrix:
[[  3   6]
 [ 20 112]]
Classification Report:
              

# GSE21815 Random SearchCV

In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier,
    AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier
)
from xgboost import XGBClassifier
import warnings

warnings.filterwarnings("ignore")

# Load datasets
df_train = pd.read_csv("ML_DEG_dataset.csv")
columns_of_interest = ['PTRF', 'PRNP', 'FLNA', 'SPARCL1', 'RHOB', 'PALLD', 'SNCA', 'CSRP1', 'TNS1',
                       'DPYSL3', 'TIMP2', 'MEF2C', 'ILK', 'ACTN1', 'TGFB1I1', 'RHOQ', 'CAV2','target']
df_train = df_train[columns_of_interest]
#df_train = df_train.iloc[:, 1:]
df_train = df_train[[col for col in df_train.columns if col != 'target'] + ['target']]
X_train = df_train.iloc[:, 0:-1]
y_train = df_train.iloc[:, -1]

df_test = pd.read_csv("GSE21815_gene_expression_renamed.csv")
df_test = df_test.iloc[:, 1:]
df_test = df_test[[col for col in df_test.columns if col != 'target'] + ['target']]
X_test = df_test.iloc[:, 0:-1]
y_test = df_test.iloc[:, -1]

# Ensure common features in both train and test sets
common_cols = set(X_train.columns).intersection(X_test.columns)
X_train_common = X_train[list(common_cols)]
X_test_common = X_test[list(common_cols)]

# Hyperparameter grid for each model
param_grids = {
    'Logistic Regression': {
        'C': np.logspace(-3, 2, 10),
        'penalty': ['l1', 'l2', 'elasticnet'],
        'solver': ['saga'],
        'max_iter': [500, 1000, 5000],
        'l1_ratio': np.linspace(0, 1, 5).tolist()
    },
    'K-Nearest Neighbors': {
        'n_neighbors': range(3, 15),
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan']
    },
    'Random Forest': {
        'n_estimators': range(100, 501, 100),
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    },
    'Gradient Boosting': {
        'n_estimators': range(100, 501, 100),
        'learning_rate': np.linspace(0.01, 0.2, 5),
        'max_depth': [3, 5, 7, 10],
        'subsample': [0.6, 0.8, 1.0],
        'min_samples_split': [2, 5, 10]
    },
    'AdaBoost': {
        'n_estimators': range(50, 301, 50),
        'learning_rate': np.linspace(0.01, 1, 5)
    },
    'Bagging': {
        'n_estimators': range(50, 301, 50),
        'max_samples': np.linspace(0.5, 1.0, 5),
        'max_features': np.linspace(0.5, 1.0, 5),
        'bootstrap': [True, False]
    },
    'Extra Trees': {
        'n_estimators': range(100, 501, 100),
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    },
    'XGBoost': {
        'n_estimators': [100, 200, 500],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    }
}

# Define models
models = {
    'Logistic Regression': LogisticRegression(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Bagging': BaggingClassifier(),
    'Extra Trees': ExtraTreesClassifier(),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss')

}

# Dictionary to store best parameters
best_params = {}

# Tune and evaluate models using RandomizedSearchCV
for name, model in models.items():
    print(f"Tuning {name}...")

    # Perform RandomizedSearchCV
    random_search = RandomizedSearchCV(model, param_grids[name],
                                       n_iter=20, cv=10,
                                       n_jobs=-1, scoring='accuracy', random_state=7)

    random_search.fit(X_train_common, y_train)

    # Store best parameters
    best_params[name] = random_search.best_params_
    print(f"Best parameters for {name}: {random_search.best_params_}\n")

    # Train model with best parameters
    best_model = random_search.best_estimator_
    best_model.fit(X_train_common, y_train)

    # Predict on test set
    predictions = best_model.predict(X_test_common)

    # Compute metrics
    test_accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted', zero_division=0)
    recall = recall_score(y_test, predictions, average='weighted', zero_division=0)
    f1 = f1_score(y_test, predictions, average='weighted', zero_division=0)
    mcc = matthews_corrcoef(y_test, predictions)

    print(f"{name} Testing Accuracy: {test_accuracy * 100:.2f}%")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-Score: {f1:.2f}")
    print(f"MCC: {mcc:.2f}")

    # Print confusion matrix and classification report
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, predictions))
    print("Classification Report:")
    print(classification_report(y_test, predictions))
    print("-" * 60)

# Print final best parameters
print("\nFinal Best Parameters for All Models:")
for model, params in best_params.items():
    print(f"{model}: {params}")

Tuning Logistic Regression...
Best parameters for Logistic Regression: {'solver': 'saga', 'penalty': 'l1', 'max_iter': 5000, 'l1_ratio': 0.5, 'C': np.float64(0.1668100537200059)}

Logistic Regression Testing Accuracy: 92.91%
Precision: 0.94
Recall: 0.93
F1-Score: 0.93
MCC: 0.52
Confusion Matrix:
[[  6   3]
 [  7 125]]
Classification Report:
              precision    recall  f1-score   support

           0       0.46      0.67      0.55         9
           1       0.98      0.95      0.96       132

    accuracy                           0.93       141
   macro avg       0.72      0.81      0.75       141
weighted avg       0.94      0.93      0.93       141

------------------------------------------------------------
Tuning K-Nearest Neighbors...
Best parameters for K-Nearest Neighbors: {'weights': 'uniform', 'n_neighbors': 3, 'metric': 'manhattan'}

K-Nearest Neighbors Testing Accuracy: 81.56%
Precision: 0.90
Recall: 0.82
F1-Score: 0.85
MCC: 0.12
Confusion Matrix:
[[  3   6]
 [ 20

# GSE44076 Grid Serach CV

In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier,
    AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier
)
from xgboost import XGBClassifier
import warnings

warnings.filterwarnings("ignore")

# Load datasets
df_train = pd.read_csv("ML_DEG_dataset.csv")
columns_of_interest = ['PTRF', 'PRNP', 'FLNA', 'SPARCL1', 'RHOB', 'PALLD', 'SNCA', 'CSRP1', 'TNS1',
                       'DPYSL3', 'TIMP2', 'MEF2C', 'ILK', 'ACTN1', 'TGFB1I1', 'RHOQ', 'CAV2','target']
df_train = df_train[columns_of_interest]
#df_train = df_train.iloc[:, 1:]
df_train = df_train[[col for col in df_train.columns if col != 'target'] + ['target']]
X_train = df_train.iloc[:, 0:-1]
y_train = df_train.iloc[:, -1]

df_test = pd.read_csv("GSE44076_gene_expression_renamed.csv")
df_test = df_test.iloc[:, 1:]
df_test = df_test[[col for col in df_test.columns if col != 'target'] + ['target']]
X_test = df_test.iloc[:, 0:-1]
y_test = df_test.iloc[:, -1]

# Ensure common features in both train and test sets
common_cols = set(X_train.columns).intersection(X_test.columns)
X_train_common = X_train[list(common_cols)]
X_test_common = X_test[list(common_cols)]

# Hyperparameter grid for each model
param_grids = {
    'Logistic Regression': {
        'C': [0.01, 0.1, 1, 10, 100],
        'penalty': ['l1', 'l2', 'elasticnet'],
        'solver': ['saga'],
        'max_iter': [500]
    },
    'K-Nearest Neighbors': {
        'n_neighbors': [3, 5, 7, 9],
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan']
    },
    'Random Forest': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    },
    'Gradient Boosting': {
        'n_estimators': [100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 1.0],
        'min_samples_split': [2, 5]
    },
    'AdaBoost': {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 1]
    },
    'Bagging': {
        'n_estimators': [50, 100, 200],
        'max_samples': [0.5, 0.75, 1.0],
        'max_features': [0.5, 0.75, 1.0],
        'bootstrap': [True, False]
    },
    'Extra Trees': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    },
     'XGBoost': {
        'n_estimators': [100, 200, 500],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    }
}

# Define models
models = {
    'Logistic Regression': LogisticRegression(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Bagging': BaggingClassifier(),
    'Extra Trees': ExtraTreesClassifier(),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

# Dictionary to store best parameters
best_params = {}

# Tune and evaluate models
for name, model in models.items():
    print(f"Tuning {name}...")

    # Perform GridSearchCV
    grid_search = GridSearchCV(model, param_grids[name], cv=10, n_jobs=-1, scoring='accuracy')
    grid_search.fit(X_train_common, y_train)

    # Store best parameters
    best_params[name] = grid_search.best_params_
    print(f"Best parameters for {name}: {grid_search.best_params_}\n")

    # Train model with best parameters
    best_model = grid_search.best_estimator_
    best_model.fit(X_train_common, y_train)

    # Predict on test set
    predictions = best_model.predict(X_test_common)

    # Compute metrics
    test_accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted', zero_division=0)
    recall = recall_score(y_test, predictions, average='weighted', zero_division=0)
    f1 = f1_score(y_test, predictions, average='weighted', zero_division=0)
    mcc = matthews_corrcoef(y_test, predictions)

    print(f"{name} Testing Accuracy: {test_accuracy * 100:.2f}%")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-Score: {f1:.2f}")
    print(f"MCC: {mcc:.2f}")

    # Print confusion matrix and classification report
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, predictions))
    print("Classification Report:")
    print(classification_report(y_test, predictions))
    print("-" * 60)

# Print final best parameters
print("\nFinal Best Parameters for All Models:")
for model, params in best_params.items():
    print(f"{model}: {params}")

Tuning Logistic Regression...
Best parameters for Logistic Regression: {'C': 0.01, 'max_iter': 500, 'penalty': 'l1', 'solver': 'saga'}

Logistic Regression Testing Accuracy: 65.54%
Precision: 0.58
Recall: 0.66
F1-Score: 0.55
MCC: 0.02
Confusion Matrix:
[[ 2 48]
 [ 3 95]]
Classification Report:
              precision    recall  f1-score   support

           0       0.40      0.04      0.07        50
           1       0.66      0.97      0.79        98

    accuracy                           0.66       148
   macro avg       0.53      0.50      0.43       148
weighted avg       0.58      0.66      0.55       148

------------------------------------------------------------
Tuning K-Nearest Neighbors...
Best parameters for K-Nearest Neighbors: {'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'uniform'}

K-Nearest Neighbors Testing Accuracy: 89.19%
Precision: 0.89
Recall: 0.89
F1-Score: 0.89
MCC: 0.76
Confusion Matrix:
[[37 13]
 [ 3 95]]
Classification Report:
              precisio

# GSE44076 Random Search CV

In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier,
    AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier
)
from xgboost import XGBClassifier
import warnings

warnings.filterwarnings("ignore")

# Load datasets
df_train = pd.read_csv("ML_DEG_dataset.csv")
columns_of_interest = ['PTRF', 'PRNP', 'FLNA', 'SPARCL1', 'RHOB', 'PALLD', 'SNCA', 'CSRP1', 'TNS1',
                       'DPYSL3', 'TIMP2', 'MEF2C', 'ILK', 'ACTN1', 'TGFB1I1', 'RHOQ', 'CAV2','target']
df_train = df_train[columns_of_interest]
#df_train = df_train.iloc[:, 1:]
df_train = df_train[[col for col in df_train.columns if col != 'target'] + ['target']]
X_train = df_train.iloc[:, 0:-1]
y_train = df_train.iloc[:, -1]

df_test = pd.read_csv("GSE44076_gene_expression_renamed.csv")
df_test = df_test.iloc[:, 1:]
df_test = df_test[[col for col in df_test.columns if col != 'target'] + ['target']]
X_test = df_test.iloc[:, 0:-1]
y_test = df_test.iloc[:, -1]

# Ensure common features in both train and test sets
common_cols = set(X_train.columns).intersection(X_test.columns)
X_train_common = X_train[list(common_cols)]
X_test_common = X_test[list(common_cols)]

# Hyperparameter grid for each model
param_grids = {
    'Logistic Regression': {
        'C': np.logspace(-3, 2, 10),
        'penalty': ['l1', 'l2', 'elasticnet'],
        'solver': ['saga'],
        'max_iter': [500, 1000, 5000],
        'l1_ratio': np.linspace(0, 1, 5).tolist()
    },
    'K-Nearest Neighbors': {
        'n_neighbors': range(3, 15),
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan']
    },
    'Random Forest': {
        'n_estimators': range(100, 501, 100),
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    },
    'Gradient Boosting': {
        'n_estimators': range(100, 501, 100),
        'learning_rate': np.linspace(0.01, 0.2, 5),
        'max_depth': [3, 5, 7, 10],
        'subsample': [0.6, 0.8, 1.0],
        'min_samples_split': [2, 5, 10]
    },
    'AdaBoost': {
        'n_estimators': range(50, 301, 50),
        'learning_rate': np.linspace(0.01, 1, 5)
    },
    'Bagging': {
        'n_estimators': range(50, 301, 50),
        'max_samples': np.linspace(0.5, 1.0, 5),
        'max_features': np.linspace(0.5, 1.0, 5),
        'bootstrap': [True, False]
    },
    'Extra Trees': {
        'n_estimators': range(100, 501, 100),
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    },
    'XGBoost': {
        'n_estimators': [100, 200, 500],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    }
}

# Define models
models = {
    'Logistic Regression': LogisticRegression(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Bagging': BaggingClassifier(),
    'Extra Trees': ExtraTreesClassifier(),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss')

}

# Dictionary to store best parameters
best_params = {}

# Tune and evaluate models using RandomizedSearchCV
for name, model in models.items():
    print(f"Tuning {name}...")

    # Perform RandomizedSearchCV
    random_search = RandomizedSearchCV(model, param_grids[name],
                                       n_iter=20, cv=10,
                                       n_jobs=-1, scoring='accuracy', random_state=7)

    random_search.fit(X_train_common, y_train)

    # Store best parameters
    best_params[name] = random_search.best_params_
    print(f"Best parameters for {name}: {random_search.best_params_}\n")

    # Train model with best parameters
    best_model = random_search.best_estimator_
    best_model.fit(X_train_common, y_train)

    # Predict on test set
    predictions = best_model.predict(X_test_common)

    # Compute metrics
    test_accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted', zero_division=0)
    recall = recall_score(y_test, predictions, average='weighted', zero_division=0)
    f1 = f1_score(y_test, predictions, average='weighted', zero_division=0)
    mcc = matthews_corrcoef(y_test, predictions)

    print(f"{name} Testing Accuracy: {test_accuracy * 100:.2f}%")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-Score: {f1:.2f}")
    print(f"MCC: {mcc:.2f}")

    # Print confusion matrix and classification report
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, predictions))
    print("Classification Report:")
    print(classification_report(y_test, predictions))
    print("-" * 60)

# Print final best parameters
print("\nFinal Best Parameters for All Models:")
for model, params in best_params.items():
    print(f"{model}: {params}")

Tuning Logistic Regression...
Best parameters for Logistic Regression: {'solver': 'saga', 'penalty': 'l1', 'max_iter': 5000, 'l1_ratio': 0.5, 'C': np.float64(0.1668100537200059)}

Logistic Regression Testing Accuracy: 84.46%
Precision: 0.84
Recall: 0.84
F1-Score: 0.84
MCC: 0.65
Confusion Matrix:
[[36 14]
 [ 9 89]]
Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.72      0.76        50
           1       0.86      0.91      0.89        98

    accuracy                           0.84       148
   macro avg       0.83      0.81      0.82       148
weighted avg       0.84      0.84      0.84       148

------------------------------------------------------------
Tuning K-Nearest Neighbors...
Best parameters for K-Nearest Neighbors: {'weights': 'uniform', 'n_neighbors': 3, 'metric': 'manhattan'}

K-Nearest Neighbors Testing Accuracy: 89.19%
Precision: 0.89
Recall: 0.89
F1-Score: 0.89
MCC: 0.76
Confusion Matrix:
[[37 13]
 [ 3 95]]
C

# GSE8671 Grid Search CV

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier,
    AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier
)
from xgboost import XGBClassifier
import warnings

warnings.filterwarnings("ignore")

# Load datasets
df_train = pd.read_csv("ML_DEG_dataset.csv")
columns_of_interest = ['PTRF', 'PRNP', 'FLNA', 'SPARCL1', 'RHOB', 'PALLD', 'SNCA',
                       'CSRP1', 'TNS1', 'DPYSL3', 'TIMP2', 'MEF2C', 'ILK', 'ACTN1', 'TGFB1I1', 'RHOQ', 'CAV2','target']
df_train = df_train[columns_of_interest]
#df_train = df_train.iloc[:, 1:]
df_train = df_train[[col for col in df_train.columns if col != 'target'] + ['target']]
X_train = df_train.iloc[:, 0:-1]
y_train = df_train.iloc[:, -1]

df_test = pd.read_csv("GSE8671_gene_expression_renamed.csv")
df_test = df_test.iloc[:, 1:]
df_test = df_test[[col for col in df_test.columns if col != 'target'] + ['target']]
X_test = df_test.iloc[:, 0:-1]
y_test = df_test.iloc[:, -1]

# Ensure common features in both train and test sets
common_cols = set(X_train.columns).intersection(X_test.columns)
X_train_common = X_train[list(common_cols)]
X_test_common = X_test[list(common_cols)]

# Hyperparameter grid for each model
param_grids = {
    'Logistic Regression': {
        'C': [0.01, 0.1, 1, 10, 100],
        'penalty': ['l1', 'l2', 'elasticnet'],
        'solver': ['saga'],
        'max_iter': [500]
    },
    'K-Nearest Neighbors': {
        'n_neighbors': [3, 5, 7, 9],
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan']
    },
    'Random Forest': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    },
    'Gradient Boosting': {
        'n_estimators': [100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 1.0],
        'min_samples_split': [2, 5]
    },
    'AdaBoost': {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 1]
    },
    'Bagging': {
        'n_estimators': [50, 100, 200],
        'max_samples': [0.5, 0.75, 1.0],
        'max_features': [0.5, 0.75, 1.0],
        'bootstrap': [True, False]
    },
    'Extra Trees': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    },
     'XGBoost': {
        'n_estimators': [100, 200, 500],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    }
}

# Define models
models = {
    'Logistic Regression': LogisticRegression(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Bagging': BaggingClassifier(),
    'Extra Trees': ExtraTreesClassifier(),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

# Dictionary to store best parameters
best_params = {}

# Tune and evaluate models
for name, model in models.items():
    print(f"Tuning {name}...")

    # Perform GridSearchCV
    grid_search = GridSearchCV(model, param_grids[name], cv=10, n_jobs=-1, scoring='accuracy')
    grid_search.fit(X_train_common, y_train)

    # Store best parameters
    best_params[name] = grid_search.best_params_
    print(f"Best parameters for {name}: {grid_search.best_params_}\n")

    # Train model with best parameters
    best_model = grid_search.best_estimator_
    best_model.fit(X_train_common, y_train)

    # Predict on test set
    predictions = best_model.predict(X_test_common)

    # Compute metrics
    test_accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted', zero_division=0)
    recall = recall_score(y_test, predictions, average='weighted', zero_division=0)
    f1 = f1_score(y_test, predictions, average='weighted', zero_division=0)
    mcc = matthews_corrcoef(y_test, predictions)

    print(f"{name} Testing Accuracy: {test_accuracy * 100:.2f}%")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-Score: {f1:.2f}")
    print(f"MCC: {mcc:.2f}")

    # Print confusion matrix and classification report
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, predictions))
    print("Classification Report:")
    print(classification_report(y_test, predictions))
    print("-" * 60)

# Print final best parameters
print("\nFinal Best Parameters for All Models:")
for model, params in best_params.items():
    print(f"{model}: {params}")

Tuning Logistic Regression...
Best parameters for Logistic Regression: {'C': 0.01, 'max_iter': 500, 'penalty': 'l1', 'solver': 'saga'}

Logistic Regression Testing Accuracy: 100.00%
Precision: 1.00
Recall: 1.00
F1-Score: 1.00
MCC: 1.00
Confusion Matrix:
[[32  0]
 [ 0 32]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        32
           1       1.00      1.00      1.00        32

    accuracy                           1.00        64
   macro avg       1.00      1.00      1.00        64
weighted avg       1.00      1.00      1.00        64

------------------------------------------------------------
Tuning K-Nearest Neighbors...
Best parameters for K-Nearest Neighbors: {'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'uniform'}

K-Nearest Neighbors Testing Accuracy: 51.56%
Precision: 0.75
Recall: 0.52
F1-Score: 0.37
MCC: 0.13
Confusion Matrix:
[[32  0]
 [31  1]]
Classification Report:
              precisi

# GSE8671 Random Search CV

In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier,
    AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier
)
from xgboost import XGBClassifier
import warnings

warnings.filterwarnings("ignore")

# Load datasets
df_train = pd.read_csv("ML_DEG_dataset.csv")
columns_of_interest = ['PTRF', 'PRNP', 'FLNA', 'SPARCL1', 'RHOB', 'PALLD', 'SNCA', 'CSRP1', 'TNS1', 'DPYSL3',
                       'TIMP2', 'MEF2C', 'ILK', 'ACTN1', 'TGFB1I1', 'RHOQ', 'CAV2','target']
df_train = df_train[columns_of_interest]
#df_train = df_train.iloc[:, 1:]
df_train = df_train[[col for col in df_train.columns if col != 'target'] + ['target']]
X_train = df_train.iloc[:, 0:-1]
y_train = df_train.iloc[:, -1]

df_test = pd.read_csv("GSE8671_gene_expression_renamed.csv")
df_test = df_test.iloc[:, 1:]
df_test = df_test[[col for col in df_test.columns if col != 'target'] + ['target']]
X_test = df_test.iloc[:, 0:-1]
y_test = df_test.iloc[:, -1]

# Ensure common features in both train and test sets
common_cols = set(X_train.columns).intersection(X_test.columns)
X_train_common = X_train[list(common_cols)]
X_test_common = X_test[list(common_cols)]

# Hyperparameter grid for each model
param_grids = {
    'Logistic Regression': {
        'C': np.logspace(-3, 2, 10),
        'penalty': ['l1', 'l2', 'elasticnet'],
        'solver': ['saga'],
        'max_iter': [500, 1000, 5000],
        'l1_ratio': np.linspace(0, 1, 5).tolist()
    },
    'K-Nearest Neighbors': {
        'n_neighbors': range(3, 15),
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan']
    },
    'Random Forest': {
        'n_estimators': range(100, 501, 100),
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    },
    'Gradient Boosting': {
        'n_estimators': range(100, 501, 100),
        'learning_rate': np.linspace(0.01, 0.2, 5),
        'max_depth': [3, 5, 7, 10],
        'subsample': [0.6, 0.8, 1.0],
        'min_samples_split': [2, 5, 10]
    },
    'AdaBoost': {
        'n_estimators': range(50, 301, 50),
        'learning_rate': np.linspace(0.01, 1, 5)
    },
    'Bagging': {
        'n_estimators': range(50, 301, 50),
        'max_samples': np.linspace(0.5, 1.0, 5),
        'max_features': np.linspace(0.5, 1.0, 5),
        'bootstrap': [True, False]
    },
    'Extra Trees': {
        'n_estimators': range(100, 501, 100),
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    },
    'XGBoost': {
        'n_estimators': [100, 200, 500],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    }
}

# Define models
models = {
    'Logistic Regression': LogisticRegression(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Bagging': BaggingClassifier(),
    'Extra Trees': ExtraTreesClassifier(),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss')

}

# Dictionary to store best parameters
best_params = {}

# Tune and evaluate models using RandomizedSearchCV
for name, model in models.items():
    print(f"Tuning {name}...")

    # Perform RandomizedSearchCV
    random_search = RandomizedSearchCV(model, param_grids[name],
                                       n_iter=20, cv=10,
                                       n_jobs=-1, scoring='accuracy', random_state=7)

    random_search.fit(X_train_common, y_train)

    # Store best parameters
    best_params[name] = random_search.best_params_
    print(f"Best parameters for {name}: {random_search.best_params_}\n")

    # Train model with best parameters
    best_model = random_search.best_estimator_
    best_model.fit(X_train_common, y_train)

    # Predict on test set
    predictions = best_model.predict(X_test_common)

    # Compute metrics
    test_accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted', zero_division=0)
    recall = recall_score(y_test, predictions, average='weighted', zero_division=0)
    f1 = f1_score(y_test, predictions, average='weighted', zero_division=0)
    mcc = matthews_corrcoef(y_test, predictions)

    print(f"{name} Testing Accuracy: {test_accuracy * 100:.2f}%")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-Score: {f1:.2f}")
    print(f"MCC: {mcc:.2f}")

    # Print confusion matrix and classification report
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, predictions))
    print("Classification Report:")
    print(classification_report(y_test, predictions))
    print("-" * 60)

# Print final best parameters
print("\nFinal Best Parameters for All Models:")
for model, params in best_params.items():
    print(f"{model}: {params}")

Tuning Logistic Regression...
Best parameters for Logistic Regression: {'solver': 'saga', 'penalty': 'l1', 'max_iter': 5000, 'l1_ratio': 0.5, 'C': np.float64(0.1668100537200059)}

Logistic Regression Testing Accuracy: 89.06%
Precision: 0.91
Recall: 0.89
F1-Score: 0.89
MCC: 0.80
Confusion Matrix:
[[25  7]
 [ 0 32]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.78      0.88        32
           1       0.82      1.00      0.90        32

    accuracy                           0.89        64
   macro avg       0.91      0.89      0.89        64
weighted avg       0.91      0.89      0.89        64

------------------------------------------------------------
Tuning K-Nearest Neighbors...
Best parameters for K-Nearest Neighbors: {'weights': 'uniform', 'n_neighbors': 3, 'metric': 'manhattan'}

K-Nearest Neighbors Testing Accuracy: 51.56%
Precision: 0.75
Recall: 0.52
F1-Score: 0.37
MCC: 0.13
Confusion Matrix:
[[32  0]
 [31  1]]
C